### PoC notebook to submit AOAI FT jobs using proxy components running on AzureML workspace

This notebook is a PoC to submit AOAI FT jobs using proxy components running on AzureML workspace. The proxy components are:
- Data Upload component: uplaods user's training and validation(optional) data to AOAI Studio and returns train and validation file id
- Submit Finetune Job: Using oai python SDK submits job to AOAI Studio for finetuning the model and polls for the terminal status of the finetune job and returns finetuned model id if the finetune job is successful.

Notes:
1. This notebook is a PoC and is not production ready.
2. The OpenAI key is stored in one of the workspace keyvault and is fetched using AzureML SDK in the component. So, the components wont work if the following workspace is not used.
3. Components use serverless compute for execution.
4. AOAI endpoint is hardcoded in the components and is not parameterized yet.

### Install dependency

In [ ]:
%pip install azure-ai-ml

### Create workspace and registry ML Client

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
    InteractiveBrowserCredential,
)
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

# Note: AOAI API Keys are stored in following workspace's KeyVault, so components would only work in this workspace
workspace_ml_client = MLClient(
    credential,
    subscription_id="72c03bf3-4e69-41af-9532-dfcdc3eefef4",
    resource_group_name="static-test-resources",
    workspace_name="test-aml-openai-ws",
)

# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml"
registry_ml_client = MLClient(credential, registry_name="finetune-dev-registry01")

### Define variables for pipeline

In [ ]:
from azure.ai.ml import Input

train_dataset = Input(type="uri_file", path="./small_train_chat.jsonl")
validation_dataset = Input(type="uri_file", path="./small_validation_chat.jsonl")
model = "gpt-35-turbo-0613"
task_type = "chat"
registered_model_name = "ft-gpt-35-turbo-0613"
n_epochs = 1
batch_size = 1
learning_rate_multiplier = 1.0


### Submit FT Job

In [ ]:
from azure.ai.ml.dsl import pipeline


# fetch the pipeline component
pipeline_component_func = registry_ml_client.components.get(
    name="aoai_proxy_pipeline", label="latest"
)

# define the pipeline job
@pipeline(
    compute="serverless"  # "serverless" value runs pipeline on serverless compute
)
def create_pipeline():
    aoai_proxy_pipeline = pipeline_component_func(
        train_dataset=train_dataset,
        validation_dataset=validation_dataset,
        model=model,
        task_type=task_type,
        registered_model_name=registered_model_name,
        n_epochs=n_epochs,
        batch_size=batch_size,
        learning_rate_multiplier=learning_rate_multiplier
    )
    return {
        "finetune_submit_output": aoai_proxy_pipeline.outputs.finetune_submit_output
    }

pipeline_object = create_pipeline()

experiment_name = "aoai_proxy_pipeline_experiment"

# submit the pipeline job
pipeline_job = workspace_ml_client.jobs.create_or_update(
    pipeline_object, experiment_name=experiment_name
)
# wait for the pipeline job to complete
workspace_ml_client.jobs.stream(pipeline_job.name)